In [1]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [2]:

from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import (PiecewisePolynomial, 
                         InverseKinematics, 
                         Sphere, 
                         Rgba, 
                         RigidTransform, 
                         RotationMatrix, 
                         Solve,
                         MathematicalProgram,
                         RollPitchYaw,
                         Cylinder)
import time
import pydrake


In [3]:
from environments import get_environment_builder

plant_builder = get_environment_builder('7DOFBINS')
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:
# def get_ik_problem_solver(plant_ik, plant_context_ik, frames, collision_free= False):
#     def solve_ik_problem(poses, q0, collision_free = collision_free):
#         tries = 3
#         q_min = plant.GetPositionLowerLimits()
#         q_max = plant.GetPositionUpperLimits()
#         q_diff =  q_max-q_min
#         ik = InverseKinematics(plant_ik, plant_context_ik)
#         #GlobalInverseKinematics(plant_ik) #InverseKinematics(plant_ik, plant_context_ik)
    
#         for pose, f in zip(poses, frames):
#             ik.AddPositionConstraint(
#                 f,
#                 [0, 0.10, 0],
#                 plant_ik.world_frame(),
#                 pose.translation()-0.02,
#                 pose.translation()+0.02,
#             )
#             # ik.AddWorldPositionConstraint(plant.GetBodyByName('ur_tool0').index(), 
#             #                               pose.translation(),
#             #                               pose.translation()-0.05,
#             #                               pose.translation()+0.05)
#             # ik.AddOrientationConstraint(
#             #     f,
#             #     RotationMatrix(),
#             #     plant_ik.world_frame(),
#             #     pose.rotation(),
#             #     0.1,
#             # )
#         if collision_free:
#             ik.AddMinimumDistanceConstraint(0.001, 0.1)
#         prog = ik.get_mutable_prog()
#         q = ik.q()
#         prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
#         prog.SetInitialGuess(q, q0)
#         result = Solve(ik.prog())
#         if result.is_success():
#                 return result.GetSolution(q)
#         return None
#     return solve_ik_problem

def show_pose(qvis, plant, plant_context, diagram, diagram_context, endeff_frame, show_body_frame = None):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    tf =plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName(endeff_frame))
    transl = tf.translation()+tf.rotation()@np.array([0,0.1,0])
    if show_body_frame is not None:
        show_body_frame(RigidTransform(tf.rotation(), transl))

def show_ik_target(pose, meshcat, name):
    h = 0.2
    if 'targ' in name:
        colors = [Rgba(1,0.5,0, 0.5), Rgba(0.5,1,0, 0.5), Rgba(0.0,0.5,1, 0.5)]
    else:
        colors = [Rgba(1,0,0, 1), Rgba(0.,1,0, 1), Rgba(0.0,0.0,1, 1)]

    rot = pose.rotation()@RotationMatrix.MakeYRotation(np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([h/2, 0,0])
    meshcat.SetObject(f"/drake/ik_target{name}/triad1",
                                   Cylinder(0.01,0.2),
                                   colors[0])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad1",RigidTransform(rot, pos))
    rot = pose.rotation()@RotationMatrix.MakeXRotation(-np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([0,h/2,0])

    meshcat.SetObject(f"/drake/ik_target{name}/triad2",
                                   Cylinder(0.01,0.2),
                                   colors[1])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad2",RigidTransform(rot, pos))
    pos= pose.translation().copy()
    rot = pose.rotation()
    pos = pos + rot@np.array([0,0,h/2])
    meshcat.SetObject(f"/drake/ik_target{name}/triad3",
                                   Cylinder(0.01,0.2),
                                   colors[2])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad3",RigidTransform(rot, pos))

# ik = InverseKinematics(plant, plant_context)
# collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)
# def eval_cons(q, c, tol):
#     return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
# col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)
# col_shunk_col =  Rgba(0.8, 0.0, 0, 0.5)    
# col_shunk_free =  Rgba(0.0, 0.8, 0.5, 0.5)

In [5]:
show_body_frame = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_acutal', 
                          )
showres = partial(show_pose, 
                  plant = plant, 
                  plant_context = plant_context, 
                  diagram = diagram, 
                  diagram_context = diagram_context,
                  endeff_frame = 'body',
                  show_body_frame=show_body_frame)

In [6]:
q = np.zeros(plant.num_positions())

sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.96705972839, min=-2.96705972839)

FloatSlider(value=0.0, description='q1', max=2.09439510239, min=-2.09439510239)

FloatSlider(value=0.0, description='q2', max=2.96705972839, min=-2.96705972839)

FloatSlider(value=0.0, description='q3', max=2.09439510239, min=-2.09439510239)

FloatSlider(value=0.0, description='q4', max=2.96705972839, min=-2.96705972839)

FloatSlider(value=0.0, description='q5', max=2.09439510239, min=-2.09439510239)

FloatSlider(value=0.0, description='q6', max=3.05432619099, min=-3.05432619099)

In [7]:
from task_space_seeding_utils import solve_ik_problem

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
#ik sliders 
show_ik_targets = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_targ', 
                          )

ik_solver = partial(solve_ik_problem, 
                    plant_ik = plant,
                    plant_context_ik = plant_context,
                    frames = [plant.GetFrameByName('body')],
                    offsets = [np.array([0,0.1,0])],
                    collision_free=False,
                    track_orientation=False)
sliders_ik = []
pos_min = np.array([-0.5,-0.5, 0.01, -np.pi,-np.pi, -np.pi])
pos_max = np.array([1.0,0.5, 1, np.pi,np.pi, np.pi])
names = ['x', 'y', 'z','rx', 'ry', 'rz']
q0 = np.zeros(7)#np.array([ 0.23294, -0.2944 , -0.36706, -1.5944 , -0.16706,  0.     ,
       # 1.64567])
_ik_positions = np.zeros(6)

for i in range(6):
    sliders_ik.append(widgets.FloatSlider(min=pos_min[i], 
                                       max=pos_max[i], 
                                       value=0.5*(pos_max[i]-pos_min[i])+pos_min[i], 
                                       description=names[i]))

def handle_slider_change_ik(change, idx):
    _ik_positions[idx] = change['new']
    rot = RotationMatrix.MakeXRotation(_ik_positions[3])@RotationMatrix.MakeYRotation(_ik_positions[4])@RotationMatrix.MakeZRotation(_ik_positions[5])

    RollPitchYaw(_ik_positions[3],
                       _ik_positions[4],
                       _ik_positions[5]).ToRotationMatrix()
    tf = RigidTransform(rot, _ik_positions[:3])
    show_ik_targets(tf)
    res = ik_solver([tf], q0)
    if res is not None:
        showres(res)

idx = 0
for slider in sliders_ik:
    slider.observe(partial(handle_slider_change_ik, idx = idx), names='value')
    idx+=1

for slider in sliders_ik:
    display(slider)

FloatSlider(value=0.25, description='x', max=1.0, min=-0.5)

FloatSlider(value=0.0, description='y', max=0.5, min=-0.5)

FloatSlider(value=0.505, description='z', max=1.0, min=0.01)

FloatSlider(value=0.0, description='rx', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='ry', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='rz', max=3.141592653589793, min=-3.141592653589793)

In [9]:
from visualization_utils import plot_points, plot_regions
from pydrake.all import VPolytope, Role
from task_space_seeding_utils import (get_cvx_hulls_of_bodies,
                                      get_AABB_cvxhull,
                                      task_space_sampler_mp,
                                      task_space_sampler)

In [10]:
#Determine task space regions of interest
geom_names = ['bin_base', 'bin_base', 'shelves_body']
model_names = ['binL', 'binR', 'shelves']
cvx_hulls_of_ROI, bodies = get_cvx_hulls_of_bodies(geom_names, model_names, plant, scene_graph, scene_graph_context)
ts_min, ts_max, cvxh_hpoly = get_AABB_cvxhull(cvx_hulls_of_ROI)
plot_regions(meshcat, cvx_hulls_of_ROI, opacity=0.2)

In [11]:
# construct task space sampler
#from task_space_seeding_utils import get_task_space_sampler
# ik_solver_S = get_ik_problem_solver(plant, 
#                                   plant_context, 
#                                   [plant.GetFrameByName('body')],
#                                   [np.array([0,0.1,0])], 
#                                   collision_free=True,
#                                   track_orientation=True)
q0  = np.zeros(7)
plant.SetPositions(plant_context, q0)
plant.ForcedPublish(plant_context)
showres(q0)
t0 = plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName("body")).translation()       

sample_handle_ts = partial(task_space_sampler_mp,
                           q0 = q0,
                           t0 = t0,
                           plant_builder = plant_builder,
                           frame_names = ['body'],
                           offsets = [np.array([0,0.1,0])],
                           cvx_hulls_of_ROI =cvx_hulls_of_ROI,
                           ts_min = ts_min,
                           ts_max = ts_max,
                           collision_free = True,
                           track_orientation = True,
                           )

In [12]:
q, t, _, res = sample_handle_ts(1000,[])

[[50, 175], [50, 158], [50, 697], [50, 149], [50, 166], [50, 335], [50, 789], [50, 545], [50, 395], [50, 690], [50, 379], [50, 363], [50, 393], [50, 344], [50, 515], [50, 506], [50, 251], [50, 424], [50, 28], [50, 439]]


100%|██████████| 50/50 [01:28<00:00,  1.76s/it]


In [14]:
res


[(array([[ 1.78284193,  1.05882945,  2.46091917,  1.58752357,  1.00860442,
          -2.0943951 ,  1.98508903],
         [-0.10585698,  1.92282782, -0.78898511, -0.02178501, -0.81038495,
          -1.68270597, -0.38722604],
         [ 1.37794971,  1.15678091,  2.41419818,  0.85808128,  1.29487004,
          -1.62387567,  0.59862245],
         [ 1.42551446, -1.28338466,  0.48369067,  0.89752582, -0.31802585,
          -2.05113297, -1.44421526],
         [-0.03042311, -1.96505061, -1.61572314,  2.0508234 , -0.75581356,
           0.68103001, -2.27981919]]),
  array([[-0.11567745,  0.37435174,  0.1888893 ],
         [ 0.75546496,  0.18774235,  0.05394997],
         [ 0.1762313 ,  0.74112466,  0.17836172],
         [ 0.0056551 , -0.50436399,  0.10755127],
         [-0.15917663,  0.6111459 ,  0.15246386]]),
  False),
 (array([[ 1.78284193,  1.05882945,  2.46091917,  1.58752357,  1.00860442,
          -2.0943951 ,  1.98508903],
         [-0.10585698,  1.92282782, -0.78898511, -0.02178501, -0

In [14]:
ik = InverseKinematics(plant, plant_context)

In [14]:
import pickle

In [12]:
q, t, _ = sample_handle_ts(300, [])

100%|██████████| 300/300 [03:44<00:00,  1.34it/s]


In [13]:
plot_points(meshcat, t, 'a', size = 0.01)

In [14]:
# import time
# for pt in q:
#     showres(pt)
#     time.sleep(0.2)

In [15]:
from utils import load_regions_from_experiment_dir

regions = load_regions_from_experiment_dir(
                    'logs/experiment_7dof_iiwa_bins_taskspace_1_1000_0.600greedy20231019114035')

In [16]:
with open('test.pkl', 'wb') as f:
    pickle.dump(regions, f)

In [17]:
from visibility_utils import point_in_regions
in_regs = []
in_regs_t = []
out_regs = []
out_regs_t = []
for tpt, pt in zip(t,q):
    if point_in_regions(pt, regions):
        in_regs.append(pt)
        in_regs_t.append(tpt)
    else:
        out_regs.append(pt)
        out_regs_t.append(tpt)

In [18]:
plot_points(meshcat, np.array(in_regs_t), 'in_regions', size=0.03, color=Rgba(0,1,0,1))

In [ ]:
len(out_regs)

300

In [17]:
for r in regions:
    qv =VPolytope(r).vertices().T
    print('r')
    for q in qv[::100]:
        showres(q)
        time.sleep(0.001)

r
r
r
r


In [18]:
qv.shape

(160420, 7)

In [18]:
import time
for pt in in_regs:
    showres(pt)
    time.sleep(0.2)

In [12]:
seed = 1
N = 1500
eps = 0.3
ts_fraction = 0.1
max_iterations_clique = 10
min_clique_size = 20
approach = 1
ap_names = ['redu', 'greedy', 'nx', 'cvxh', 'cvxh_ell']
extend_cliques = False

require_sample_point_is_contained = True
iteration_limit = 1
configuration_space_margin = 2.e-3
termination_threshold = -1
num_collision_infeasible_samples = 15
relative_termination_threshold = 0.02

pts_coverage_estimator = 5000
cfg = {'seed': seed,
        'N': N,
        'eps': eps,
        'use_ts': True,
        'ts_fraction': ts_fraction,
        'max_iterations_clique': max_iterations_clique,
        'min_clique_size': min_clique_size,
        'approach': approach,
        'extend_cliques': extend_cliques,
        'require_sample_point_is_contained':require_sample_point_is_contained,
        'iteration_limit': iteration_limit,
        'configuration_space_margin':configuration_space_margin,
        'termination_threshold':termination_threshold,
        'num_collision_infeasible_samples':num_collision_infeasible_samples,
        'relative_termination_threshold':relative_termination_threshold,
        'pts_coverage_estimator':pts_coverage_estimator}

In [13]:
from visibility_utils import (get_col_func, 
                              get_sample_cfree_handle,
                              get_coverage_estimator,
                              vgraph)
from region_generation import SNOPT_IRIS_ellipsoid_parallel
from pydrake.all import SceneGraphCollisionChecker
from visibility_logging import CliqueApproachLogger
from visibility_clique_decomposition import VisCliqueInflation

In [14]:
q_min = plant.GetPositionLowerLimits()*1
q_max =  plant.GetPositionUpperLimits()*1

col_func_handle_ = get_col_func(plant, plant_context)
sample_cfree = get_sample_cfree_handle(q_min,q_max, col_func_handle_)
estimate_coverage = get_coverage_estimator(sample_cfree, pts = pts_coverage_estimator)

snopt_iris_options = IrisOptions()
snopt_iris_options.require_sample_point_is_contained = require_sample_point_is_contained
snopt_iris_options.iteration_limit = iteration_limit
snopt_iris_options.configuration_space_margin = configuration_space_margin
#snopt_iris_options.max_faces_per_collision_pair = 60
snopt_iris_options.termination_threshold = termination_threshold
#snopt_iris_options.q_star = np.zeros(3)
snopt_iris_options.num_collision_infeasible_samples = num_collision_infeasible_samples
snopt_iris_options.relative_termination_threshold = relative_termination_threshold
robot_instances = [plant.GetModelInstanceByName("iiwa"), plant.GetModelInstanceByName("wsg")]

checker = SceneGraphCollisionChecker(model = diagram.Clone(), 
                    robot_model_instances = robot_instances,
                    distance_function_weights =  [1] * plant.num_positions(),
                    #configuration_distance_function = _configuration_distance,
                    edge_step_size = 0.125)

def sample_handle_joint(N, M, regions, frac_ts_samples = ts_fraction, collision_free=True):
    N_ts = int(frac_ts_samples*N)
    N_c = N-N_ts
    pts_q_ts, pts_t, is_full_ts = sample_handle_ts(N_ts, regions, collision_free=collision_free)
    pts_q_c, is_full = sample_cfree(N_c, M, regions)
    return np.concatenate((pts_q_ts, pts_q_c), axis=0), is_full

vgraph_handle = partial(vgraph, checker = checker, parallelize = True) 


INFO:drake:Allocating contexts to support 20 parallel queries given omp_num_threads 20 omp_max_threads 20 and omp_thread_limit 2147483647 OpenMP enabled in build? true


In [15]:
clogger = CliqueApproachLogger(f"7dof_iiwa_bins",f"{ap_names[approach]}", estimate_coverage=estimate_coverage, cfg_dict=cfg)
iris_handle = partial(SNOPT_IRIS_ellipsoid_parallel,
                        region_obstacles = [],
                        logger = clogger, 
                        plant_builder = plant_builder,
                        snoptiris_options = snopt_iris_options,
                        estimate_coverage = estimate_coverage,
                        coverage_threshold = 1- eps)

vcd = VisCliqueInflation(N, 
                    eps,
                    max_iterations=max_iterations_clique,
                    sample_cfree = sample_handle_joint,
                    col_handle= col_func_handle_,
                    build_vgraph=vgraph_handle,
                    iris_w_obstacles=iris_handle,
                    verbose = True,
                    logger=clogger,
                    approach=approach,
                    extend_cliques=extend_cliques,
                    min_clique_size = min_clique_size
                    )
regs = vcd.run()

logdir created
[16:03:25] [VisCliqueDecomp] Attempting to cover 70.0 '%' of Cfree 


100%|██████████| 150/150 [00:18<00:00,  7.97it/s]


[16:06:21] [VisCliqueDecomp] Found  9  cliques


INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking wsg::collision against shelves::top has already required 150 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against shelves::top after 161 counter-example searches.
INFO:drake: Checking wsg::collision against shelves::top has already required 150 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against shelves::top after 157 counter-example searches.
INFO:drake: Checking wsg::collision against shelves::top has already required 150 counter-example searches; st

[SNOPT IRIS Worker]: Region:0 / 1
[SNOPT IRIS Worker]: Region:0 / 1
[SNOPT IRIS Worker]: Region:0 / 1


...

INFO:drake: Checking wsg::collision against shelves::top has already required 150 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against shelves::top after 151 counter-example searches.


[SNOPT IRIS Worker]: Region:0 / 1


INFO:drake: Checking wsg::collision against binR::left has already required 150 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against binR::left after 165 counter-example searches.


[SNOPT IRIS Worker]: Region:0 / 1
[SNOPT IRIS Worker]: Region:0 / 1
[SNOPT IRIS Worker]: Region:0 / 1
[SNOPT IRIS Worker]: Region:0 / 1


100%|██████████| 150/150 [00:21<00:00,  6.95it/s]


(100, 7)

In [17]:
pts_q, pts_t, _ = sample_handle_ts(100,[])
plot_points(meshcat, pts_t.squeeze(), 'tspts', size=0.01)

100%|██████████| 100/100 [00:10<00:00,  9.67it/s]


In [32]:
from dijkstraspp import DijkstraSPPsolver
def dm(q):
    return q
dspp = DijkstraSPPsolver(regions, dm)

[DijkstraSPP] Pre-Building adjacency matrix  0 / 107
[DijkstraSPP] Pre-Building adjacency matrix  10 / 107
[DijkstraSPP] Pre-Building adjacency matrix  20 / 107
[DijkstraSPP] Pre-Building adjacency matrix  30 / 107
[DijkstraSPP] Pre-Building adjacency matrix  40 / 107
[DijkstraSPP] Pre-Building adjacency matrix  50 / 107
[DijkstraSPP] Pre-Building adjacency matrix  60 / 107
[DijkstraSPP] Pre-Building adjacency matrix  70 / 107
[DijkstraSPP] Pre-Building adjacency matrix  80 / 107
[DijkstraSPP] Pre-Building adjacency matrix  90 / 107
[DijkstraSPP] Pre-Building adjacency matrix  100 / 107
[DijkstraSPP] Pre-Building d-adjacency matrix  0 / 5420
[DijkstraSPP] Pre-Building d-adjacency matrix  1000 / 5420
[DijkstraSPP] Pre-Building d-adjacency matrix  2000 / 5420
[DijkstraSPP] Pre-Building d-adjacency matrix  3000 / 5420
[DijkstraSPP] Pre-Building d-adjacency matrix  4000 / 5420


In [ ]:
import pickle
with open('dspp_bins_gcs.pkl', 'wb') as f:
    pickle.dump(dspp)

In [31]:
import time
for pt in q:
    showres(pt)
    time.sleep(0.01)

In [17]:
for i, v in enumerate(vp):
    plot_points(meshcat, v.vertices().T, f"vp{i}")

In [74]:
ids = Inspector.GetGeometries(binL_body)
shapes = [Inspector.GetShape(id) for id in ids]

BodyIndex(15)

In [78]:
from pydrake.all import PolygonSurfaceMesh

In [66]:
plant.GetBodyByName('shelves_body', plant.GetModelInstanceByName('shelves'))
plant.GetBodyByName('top_and_bottom', plant.GetModelInstanceByName('shelves'))


<RigidBody name='top_and_bottom' index=16 model_instance=4>

In [46]:
Inspector.GetFrameId()

TypeError: GetFrameId(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.geometry.SceneGraphInspector, geometry_id: pydrake.geometry.GeometryId) -> pydrake.geometry.FrameId

Invoked with: <pydrake.geometry.SceneGraphInspector object at 0x7f524109b770>

In [31]:
from pydrake.all import GlobalInverseKinematics

In [51]:
Inspector.Get()

TypeError: GetGeometryIdByName(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.geometry.SceneGraphInspector, frame_id: pydrake.geometry.FrameId, role: pydrake.geometry.Role, name: str) -> pydrake.geometry.GeometryId

Invoked with: <pydrake.geometry.SceneGraphInspector object at 0x7f524109b770>

In [17]:
g_ik = GlobalInverseKinematics(plant)

In [22]:
g_ik.AddWorldPositionConstraint(plant.GetBodyByName('base').index(), 
                                np.zeros(3),
                                tf.translation()-0.05,
                                tf.translation()+0.05,)

NameError: name 'tf' is not defined

In [ ]:
def get_gik_problem_solver(plant_ik, plant_context_ik, body_indices, collision_free= False):
    def solve_gik_problem(poses, q0, collision_free = collision_free):
        q_min = plant.GetPositionLowerLimits()
        q_max = plant.GetPositionUpperLimits()
        q_diff =  q_max-q_min
        gik = GlobalInverseKinematics(plant_ik)
        #GlobalInverseKinematics(plant_ik) #InverseKinematics(plant_ik, plant_context_ik)
    
        for pose, b in zip(poses, body_indices):
            gik.AddWorldPositionConstraint(
                b,
                np.zeros(3),
                plant_ik.world_frame(),
                pose.translation()-0.05,
                pose.translation()+0.05,
            )
            # ik.AddWorldPositionConstraint(plant.GetBodyByName('ur_tool0').index(), 
            #                               pose.translation(),
            #                               pose.translation()-0.05,
            #                               pose.translation()+0.05)
            gik.AddOrientationConstraint(
                f,
                RotationMatrix(),
                plant_ik.world_frame(),
                pose.rotation(),
                0.01,
            )
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())
        if result.is_success():
                return result.GetSolution(q)
        return None
    return solve_ik_problem

In [18]:
dir(g_ik)

['AddPostureCost',
 'AddWorldOrientationConstraint',
 'AddWorldPositionConstraint',
 'AddWorldRelativePositionConstraint',
 'Options',
 'ReconstructGeneralizedPositionSolution',
 'SetInitialGuess',
 '__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_pybind11_del_orig',
 'body_position',
 'body_rotation_matrix',
 'get_mutable_prog',
 'prog']

In [27]:
GlobalInverseKinematics.Options()

GlobalInverseKinematics.Options(num_intervals_per_half_axis=2, approach=Approach.kBilinearMcCormick, interval_binning=IntervalBinning.kLogarithmic, linear_constraint_only=False)

FloatSlider(value=0.0, description='x', max=1.0, min=-0.5)

FloatSlider(value=0.0, description='y', max=0.5, min=-0.5)

FloatSlider(value=0.01, description='z', max=1.0, min=0.01)

FloatSlider(value=0.0, description='rx', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='ry', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=0.0, description='rz', max=3.141592653589793, min=-3.141592653589793)

In [ ]:
# start = np.array([-2.06706, -0.3944 ,  1.43294,  0.9056 ,  0.53294, -0.8944 ,
#         0.74567])
# end = np.array([ 1.23294, -0.1944 ,  0.     ,  1.6056 ,  1.43294,  1.1056 ,
#         0.24567])

poi =np.array([
    #[-2.06706, -0.3944 ,  1.43294,  0.9056 ,  0.53294, -0.8944 ,
     #   0.74567],
        [ 1.83294, -0.0944 , -0.06706, -1.2944 ,  1.53294,  1.4056 ,
        0.34567],
        [ 1.23294, -0.1944 ,  0.     ,  1.6056 ,  1.43294,  1.1056 ,
        0.24567],
        [ 1.23294, -0.1944 ,  0.     ,  1.1056 ,  1.43294,  1.1056 ,
       -0.15433],
       [ 1.23294, -0.4944 , -0.06706,  2.0056 ,  1.23294,  1.3056 ,
        0.94567],
        
        [ 1.83294,  0.3056 , -0.06706, -1.3944 ,  1.53294,  1.4056 ,
       -0.05433],
       [ 1.63294,  0.7056 ,  0.23294, -1.6944 ,  1.53294,  1.2056 ,
       -0.65433]])
showres(poi[0])

In [ ]:
import os 
'experiment_7dof_iiwa__1_1500_0.300greedy20230908201834' in os.listdir('logs')

In [ ]:
import pickle
path = 'logs_icra_paper/experiment_7dof_iiwa__1_1500_0.300greedy20230908201834'
with open(path+'/data/it_2.pkl', 'rb') as f:
    data = pickle.load(f)

regions = []
for ga,gb in zip(data['ra'], data['rb']):
    for a,b in zip(ga,gb):
        regions.append(HPolyhedron(a,b))
regions = regions[:43]

In [ ]:
from visibility_utils import point_in_regions
for p in poi:
    print(point_in_regions(p, regions))

In [ ]:
from region_generation import SNOPT_IRIS_obstacles
def estimate_coverage(r):
    return 0
snopt_iris_options = IrisOptions()
snopt_iris_options.require_sample_point_is_contained = True
snopt_iris_options.iteration_limit = 6
snopt_iris_options.configuration_space_margin = 2e-3
#snopt_iris_options.max_faces_per_collision_pair = 60
snopt_iris_options.termination_threshold = -1
#snopt_iris_options.q_star = np.zeros(3)
snopt_iris_options.num_collision_infeasible_samples = 15
snopt_iris_options.relative_termination_threshold = 0.02

iris_handle = partial(SNOPT_IRIS_obstacles, 
                        logger = None, 
                        plant = plant, 
                        context = diagram_context,
                        snoptiris_options = snopt_iris_options,
                        estimate_coverage = estimate_coverage,
                        coverage_threshold = 1)


showres(poi[4,:])

# 213  054
seq = [2,1,3,5,4,0,2] 
# r_poi,_ = iris_handle(poi, [], [])
#r_poi = [r1[0],r2[0]]
# with open('rpoi_3shelf.pkl', 'wb') as f:
#     ra = [r.A() for r in r_poi]
#     rb = [r.b() for r in r_poi]
#     pickle.dump([ra, rb], f)

In [ ]:
import pickle
with open('tmp/rpoi_3shelf.pkl', 'rb') as f:
    rpoilist = pickle.load(f)

r_poi = []
for ra, rb in zip(rpoilist[0], rpoilist[1]):
    r_poi.append(HPolyhedron(ra,rb))
    ra = [r.A() for r in r_poi]
    rb = [r.b() for r in r_poi]
    #pickle.dump([ra, rb], f)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from visibility_utils import generate_distinct_colors

regs = regions+r_poi
connectivity_graph = nx.Graph()
for idx in range(len(regs)):
    connectivity_graph.add_node(idx)


for idx1 in range(len(regs)):
    for idx2 in range(idx1 +1, len(regs)):
        r1 = regs[idx1]
        r2 = regs[idx2]
        if r1.IntersectsWith(r2):
            connectivity_graph.add_edge(idx1,idx2)

largest_con_comp = list(list(nx.connected_components(connectivity_graph))[0])
regions_connected_component = [regs[i] for i in largest_con_comp]

fig = plt.figure(figsize=(10,10))
hues = generate_distinct_colors(2+1)[1:]
colors = []
for g, h in zip([regions, r_poi], hues):
    colors += [h]*len(g)

nx.draw_spring(connectivity_graph, 
                with_labels = True, 
                node_color = colors)
plt.title(f"connectivity")

def densify_waypoints(waypoints_q, densify = 200):
    dists = []
    dense_waypoints = []
    for idx in range(len(waypoints_q[:-1])):
        a = waypoints_q[idx]
        b = waypoints_q[idx+1]
        t = np.linspace(1,0, 10)
        locs_endeff = []
        dists_endeff = []
        for tval in t:
            a = a*tval + b*(1-tval)
            qa = a#Ratfk.ComputeQValue(ta, np.zeros(7))
            #showres(qa)
            #time.sleep(0.1)            
            plant.SetPositions(plant_context, qa)
            tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(9)))
            tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.13
            locs_endeff.append(tf)
        for i in range(len(locs_endeff)-1):
            dists_endeff.append(np.linalg.norm(locs_endeff[i]- locs_endeff[i+1]))
        d = np.sum(dists_endeff)
        #print(d * densify)
        t = np.linspace(1,0,int(densify*d))
        for tval in t:
            dense_waypoints.append(waypoints_q[idx]*tval + waypoints_q[idx+1]*(1-tval))
    return dense_waypoints

def plot_endeff_traj_s(dense,name = '', color = (1,0,0,1)):
    plot_endeff_traj(np.array([Ratfk.ComputeQValue(w,np.zeros(7)) for w in dense]), name, color)

def plot_endeff_traj(dense_waypoints, name = '', color= (1,0,0,1)):
    color = Rgba(color[0], color[1], color[2], color[3])
    start_idx = 0
    for i, qa in enumerate(dense_waypoints):
        #qa = Ratfk.ComputeQValue(ta, np.zeros(7))
        #showres(qa)
        #time.sleep(0.1)            
        plant.SetPositions(plant_context, qa)
        tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(9)))
        tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.13

        meshcat.SetObject(f"/iris/points/traj/{name}/{i+start_idx}",
                               Sphere(0.01),
                               color)

        meshcat.SetTransform(f"/iris/points/traj/{name}/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  tf))

In [ ]:
def conversion_dummy(q):
    return q
import dijkstraspp
dspp = dijkstraspp.DijkstraSPPsolver(regions + r_poi, conversion_dummy)
# import pickle
# dsppname = "7dofnew_dspp_small.pkl"
# with open(dsppname, 'rb') as f:
#     dspp = pickle.load( f)
import pickle
dsppname = "tmp/7dofnew_dspp_small.pkl"
with open(dsppname, 'wb') as f:
    pickle.dump(dspp, f)

In [ ]:
colors = [(1,0,0,1),(1,1,0,1),
(0,1,0,1),
(0,1,1,1),
(0,0,1,1),
(1,0,1,1),
]

meshcat.Delete("/iris/points/traj/")
for idx,i in enumerate(seq[:-1]):
    wp, dist = dspp.solve(poi[i,:], poi[seq[idx+1],:], refine_path = True)
    wpd = densify_waypoints(wp)
    plot_endeff_traj(wpd, str(i), colors[i])




In [ ]:
from pydrake.all import Mesh
a = Mesh('display_signs/7dof_sign_arxiv.gltf')
meshcat.SetObject('/instructionsign', a)
meshcat.SetTransform('/instructionsign',RigidTransform(
                            RotationMatrix.MakeZRotation(-np.pi/2)@RotationMatrix.MakeXRotation(-np.pi/2), 
                            np.array([0, 10 , 0])))

In [ ]:
frame_time = 1/32.0
cur_time = 0
meshcat.SetProperty(f"/Grid", "visible", False)
meshcat.SetProperty(f"/drake/visualizer/shunk", "visible", False)
meshcat.StartRecording()
animation = meshcat.get_mutable_recording()
for i in range(20):
    wp, dist = dspp.solve(poi[seq[(i)%len(seq)],:], poi[seq[(i+1)%len(seq)],:], refine_path = True)
    wpd = densify_waypoints(wp, 50)
    for qa in wpd:
        showres(qa)
        diagram_context.SetTime(cur_time)
        if col_func_handle(qa):
            print('col')
        cur_time+=frame_time
        time.sleep(frame_time)
meshcat.StopRecording()
animation.set_autoplay(True)
meshcat.PublishRecording()
with open("static_htmls/7DOF_IIWA_arxiv.html", "w+") as f:
    f.write(meshcat.StaticHtml())

In [ ]:
def plot_endeff_traj_s(dense,name = '', color = (1,0,0,1)):
    plot_endeff_traj(np.array([Ratfk.ComputeQValue(w,np.zeros(7)) for w in dense]), name, color)

def plot_endeff_traj(dense_waypoints, name = '', color= (1,0,0,1)):
    color = Rgba(color[0], color[1], color[2], color[3])
    start_idx = 0
    for i, qa in enumerate(dense_waypoints):
        #qa = Ratfk.ComputeQValue(ta, np.zeros(7))
        #showres(qa)
        #time.sleep(0.1)            
        plant.SetPositions(plant_context, qa)
        tf_tot= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(9)))
        tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.13

        meshcat.SetObject(f"/iris/points/traj/{name}/{i+start_idx}",
                               Sphere(0.01),
                               color)

        meshcat.SetTransform(f"/iris/points/traj/{name}/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  tf))